In [ ]:
# Setting
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
#1 데이터 로딩
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
#2 Config
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CUDA = torch.cuda.is_available()

# Hyper parameters
num_classes = 10

In [ ]:
#3 Build the model (1) -- 큰 의미 없는 모델
class CNN(nn.Module):
def __init__(self, num_classes=10):
super(CNN, self).__init__()
self.block1 = nn.Sequential(
# kernel_size = (5X5)
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=16, momentum=0.1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=16, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=32, momentum=0.1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=32, momentum=0.1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
self.fc = nn.Linear(8*8*32, num_classes)

def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.view(x.size(0), -1)
        output = self.fc(x)
return output

# model = CNN(num_classes=num_classes).to(device)
# 이후 아래 코드를 이용하여 학습을 진행한 결과 74%의 정확도에 머물렀다. (Epochs=10)


#3 Build the model (2) -- ResNet 이용
class ConvBlock(nn.Module):

def __init__(self, in_channel, out_channel, stride=1):
super(ConvBlock, self).__init__()
# Resnet50의 경우 block 내에 conv+bn 세트가 3개인데 simple하게 2개만 쌓았다.
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
self.bn1 = nn.BatchNorm2d(num_features=out_channel)
self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=3, stride=1, padding=1, bias=False)
self.bn2 = nn.BatchNorm2d(num_features=out_channel)

# Skip connection을 위해 shortcut을 생성한다.
        # 이 Block은 Identity Block이 아니라 Convolutional Block으로 설계하였기 때문에
        # 아래처럼 작은 1X1 Convolutional Layer를 추가한다.
        self.shortcut = nn.Sequential()

# 첫 MakeLayer에서는 아래와 같은 shortcut을 넣지 않는다는 의미이다.
        # 그 다음부터 channel이 확장되거나 stride를 1보다 크게 하여 (점점 깊어지면서) 정보를 압축할 때 shortcut을 이용하겠다는 의미.
        if stride != 1 or in_channel != out_channel:
self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

def forward(self, x):
# 2번의 Convolutional Layer + Batch Norm Layer를 통과한 뒤,
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

# 활성화 함수를 적용하기 전에 shortcut을 더해준다.
        out = out + self.shortcut(x)
        out = F.relu(out)
        out = out.cuda()
return out

In [ ]:
class ResNet(nn.Module):
def __init__(self, block, num_blocks, num_classes=10):
super(ResNet, self).__init__()
self.in_channel = 64

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
self.bn1 = nn.BatchNorm2d(num_features=64)
self.layer1 = self.MakeLayer(block=block, out_channel=64, num_blocks=num_blocks[0], stride=1)
self.layer2 = self.MakeLayer(block=block, out_channel=128, num_blocks=num_blocks[1], stride=2)
self.layer3 = self.MakeLayer(block=block, out_channel=256, num_blocks=num_blocks[2], stride=2)
self.layer4 = self.MakeLayer(block=block, out_channel=512, num_blocks=num_blocks[3], stride=2)
self.linear = nn.Linear(512, num_classes)

def MakeLayer(self, block, out_channel, num_blocks, stride):
# Convblock으로 형성한 Layer들을 nn.Sequential을 이용하여 구성한다.
        # stride가 1일 경우 아래 strides = [1, 1]
        strides = [stride] + [1]*(num_blocks-1)
        layers = []

# Block 형성이 끝난 이후에는 채널 확장을 위해 self.in_channel = out_channel로 확장된 채널 수로 업데이트를 해준다.
        for stride in strides:
            layers.append(block(self.in_channel, out_channel, stride))
self.in_channel = out_channel
return nn.Sequential(*layers)

def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = out.cuda()
return out


learning_rate = 0.01
# 0.001로 낮춤

model = ResNet(block=ConvBlock, num_blocks=[2,2,2,2], num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



if CUDA:
    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))


#4 Train the network
def Train(epochs=1):

for epoch in range(epochs):

        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
# get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

# zero the parameter gradients
            optimizer.zero_grad()

# forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# print statistics
            # item(): retrieve a single value
            running_loss += loss.item()

            print_op = 500

            if (i + 1) % print_op == 0:    # print every `print_op` mini-batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / print_op))
                running_loss = 0.0

    print('Finished Training')


Train(epochs=15)
# learning_rate = 0.01로 Epoch 15까지
# learning_rate = 0.001로 Epoch 5 추가


#5 Test
correct = 0
total = 0

class_correct = list(0.0 for i in range(10))
class_total = list(0.0 for i in range(10))

In [ ]:
with torch.no_grad():
for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1) # prediction
        c = (predicted == labels).squeeze()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# torch.save(model, 'C:/Users/YY/Documents/Winter Project/Spring/epoch20.pt')
# Resnet18: Test 정확도는 78% 수준(과적합 됨)
# Data Augmentation을 통해 더욱 Robust한 모델을 구성할 수 있을 것